In [37]:
import pandas as pd
import itertools
import numpy as np

In [51]:
df = pd.read_excel('Seleccion.xlsx').iloc[:30]
df = df.fillna('N/A')
df = df.rename(columns={
    "generative_objective falcon": "generative_objective_falcon",
    "extractive_objective falcon": "extractive_objective_falcon"
})

In [52]:
all_cols = df.columns.tolist()
model_names = set()

for col in all_cols:
    if (col.startswith('generative_objective_') or 
        col.startswith('extractive_objective_') or 
        col.startswith('generative_objective ') or 
        col.startswith('extractive_objective ')):
        
        if '_deep' not in col and 'SCORE' not in col:
            if col.startswith('generative_objective_'):
                model_name = col.replace('generative_objective_', '')
            elif col.startswith('extractive_objective_'):
                model_name = col.replace('extractive_objective_', '')
            elif col.startswith('generative_objective '):
                model_name = col.replace('generative_objective ', '')
            elif col.startswith('extractive_objective '):
                model_name = col.replace('extractive_objective ', '')
            
            model_names.add(model_name.strip())

models = sorted(list(model_names))
models


['falcon',
 'gemma2-9b',
 'gemma3-4b',
 'gpt-4o-mini',
 'llama3.1-8b',
 'llama4-16x17b',
 'mistral-7b',
 'mixtral-8x22b',
 'qwen3-32b',
 'qwen3-8b']

In [54]:
cols_to_check = []
for model in models:
    for task in ["generative_objective_", "extractive_objective_"]:
        col_name = f"{task}{model}"
        cols_to_check.append(col_name)

# remove any rows where at least one of the cols_to_check has a N/A, np.nan, "/" or empty string
filtered_df = df.copy()
for col in cols_to_check:
    filtered_df = filtered_df[~filtered_df[col].isin(['N/A', np.nan, '/', ''])]

#
nr_eval = 10
filtered_df = filtered_df.sample(n=nr_eval, random_state=42).reset_index(drop=True)

In [55]:
for col in cols_to_check:
    assert all(~filtered_df[col].isin(['N/A', np.nan, '/', ''])), f"Column {col} has invalid entries."

In [56]:
filtered_df[cols_to_check]

,generative_objective_falcon,extractive_objective_falcon,generative_objective_gemma2-9b,extractive_objective_gemma2-9b,generative_objective_gemma3-4b,extractive_objective_gemma3-4b,generative_objective_gpt-4o-mini,extractive_objective_gpt-4o-mini,generative_objective_llama3.1-8b,extractive_objective_llama3.1-8b,generative_objective_llama4-16x17b,extractive_objective_llama4-16x17b,generative_objective_mistral-7b,extractive_objective_mistral-7b,generative_objective_mixtral-8x22b,extractive_objective_mixtral-8x22b,generative_objective_qwen3-32b,extractive_objective_qwen3-32b,generative_objective_qwen3-8b,extractive_objective_qwen3-8b
0,El objetivo es la contratación de servicios té...,el servicio de mantenimiento de zonas verdes,El objetivo es la contratación de dos servicio...,"Como se ha indicado en la memoria técnica, el ...",El objetivo es la coordinación de seguridad y ...,El presente expediente técnico se redacta para...,El objetivo es la contratación de dos servicio...,"""la contratación de dos servicios/asistencias ...",El objetivo es la coordinación de seguridad y ...,Como no se menciona explícitamente el objeto d...,El objetivo es coordinar la seguridad y salud ...,/ \n\n(No se encontró una descripción explícit...,El objetivo es contratar dos servicios técnico...,"Como se ha indicado en la memoria técnica, el ...",El objetivo es contratar dos servicios/asisten...,"Como se ha indicado en la memoria técnica, el ...",El objetivo es la prestación de dos servicios ...,"Como se ha indicado en la memoria técnica, el ...",El objetivo es la coordinación de seguridad y ...,"[Como se ha indicado en la memoria técnica, el..."
1,El objetivo es realizar el saneamiento y pintu...,El saneamiento y pintura de paredes y techos d...,El objetivo es realizar trabajos de saneamient...,el saneamiento y pintura de paredes y techos d...,El objetivo es el saneamiento y pintura de par...,El saneamiento y pintura de paredes y techos d...,El objetivo es realizar las obras de pintura y...,La elaboración del presente Pliego tiene por o...,El objetivo es realizar la pintura y saneamien...,Constituye el objeto de este pliego el estable...,El objetivo es realizar obras de pintura y san...,"El objeto del contrato es: \n""las obras de pin...",El objetivo es realizar la saneamiento y pintu...,La elaboración del presente Pliego tiene por o...,El objetivo es establecer las condiciones técn...,"""La elaboración del presente Pliego tiene por ...",El objetivo es la ejecución de obras de pintur...,Constituye el objeto de este pliego el estable...,El objetivo es la realización de obras de pint...,It seems like your message is getting stuck in...
2,"El objetivo es llevar a cabo el mantenimiento,...",El objeto de este pliego es establecer las car...,"El objetivo es el mantenimiento, reparación y ...","el mantenimiento, la reparación y la construcc...",El objetivo es la prestación de un servicio de...,"El mantenimiento, la reparación y la construcc...","El objetivo es el mantenimiento, reparación y ...",la prestación del servicio de mantenimiento de...,"El objetivo es realizar el mantenimiento, repa...","El mantenimiento, la reparación y la construcc...",El objetivo es prestar el servicio de mantenim...,"""el mantenimiento de las estaciones de aforos ...",El objetivo es contratar el servicio de manten...,"El mantenimiento, la reparación y la construcc...","El objetivo es llevar a cabo el mantenimiento,...","""El mantenimiento, la reparación y la construc...","El objetivo es el mantenimiento, reparación y ...",[Este Pliego tiene por objeto establecer las c...,"El objetivo es el mantenimiento, reparación y ...",Entiendo que estás intentando construir una fr...
3,El objetivo es suministrar un equipo completo ...,El objeto del presente expediente de contratac...,El objetivo es el suministro de un Equipo de P...,Suministro de un Equipo de Pruebas Cardiorresp...,El objetivo es el Suministro de un Equipo de P...,Suministro de Equipo Pruebas Cardiorrespirator...,El objetivo

In [58]:
tasks = {
    'generative': {
        'prefix': 'generative_objective_'
    },
    'extractive': {
        'prefix': 'extractive_objective_'
    }
}

In [ ]:
comparison_list = []
id_counter = 1

for index, row in filtered_df.iterrows():
    place_id = row.get('place_id')
    source_context = row.get('texto_tecnico', '') 
    
    for task_name, task_info in tasks.items():
        reference_text = row.get('title') 

        for model_a, model_b in itertools.combinations(models, 2):
            output_a_col = task_info['prefix'] + model_a
            output_b_col = task_info['prefix'] + model_b

            output_a = row.get(output_a_col, 'N/A')
            output_b = row.get(output_b_col, 'N/A')
            
            text_list = [output_a, output_b]
            if output_a == 'N/A' or pd.isna(output_a):
                text_list[0] = "/"
            if output_b == 'N/A' or pd.isna(output_b):
                text_list[1] = "/"
            
            text_list = "||".join(text_list)
            
            comparison_list.append({
                'id': id_counter,
                'place_id': place_id,
                'task_type': task_name,
                'reference_text': reference_text, 
                'source_document_context': source_context, 
                'link_pdf': row.get('link', ''), #row.get('path_tecnico', ''),
                'text_optionA': output_a,
                'text_optionB': output_b,
                'item_a_model': model_a,
                'item_b_model': model_b,
                'text': text_list,
            })
            id_counter += 1

comparison_df = pd.DataFrame(comparison_list)

final_cols = [
    'id', 'place_id', 'task_type', 
    'reference_text', 'source_document_context', 'link_pdf',
    'item_a_model', 'text', 
    'item_b_model', 'text_optionA', 'text_optionB'
]

final_df_eval = comparison_df[final_cols]
final_df_eval.to_csv("/export/usuarios_ml4ds/lbartolome/Repos/patchwork/experiments/objective_extractor/potato/tasks/phase1/data_files/Selected_Comparisons.csv", index=False, encoding='utf-8')
final_df_eval.head(2)

,id,place_id,task_type,reference_text,source_document_context,link_pdf,item_a_model,text,item_b_model,text_optionA,text_optionB
0,1,https://contrataciondelestado.es/sindicacion/l...,generative,Control y Gestión de los Contratos de Energía ...,Código Seguro de Verificación\nIV7WGLC43AU52SM...,https://contrataciondelestado.es/wps/poc?uri=d...,falcon,El objetivo es la contratación de servicios té...,gemma2-9b,El objetivo es la contratación de servicios té...,El objetivo es la contratación de dos servicio...
1,2,https://contrataciondelestado.es/sindicacion/l...,generative,Control y Gestión de los Contratos de Energía ...,Código Seguro de Verificación\nIV7WGLC43AU52SM...,https://contrataciondelestado.es/wps/poc?uri=d...,falcon,El objetivo es la contratación de servicios té...,gemma3-4b,El objetivo es la contratación de servicios té...,El objetivo es la coordinación de seguridad y ...


In [48]:
final_df_eval.place_id.iloc[0]

'https://contrataciondelestado.es/sindicacion/licitacionesPerfilContratante/14588585'